In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import pandas as pd
import numpy as np
matplotlib.style.use('ggplot')
from sklearn.model_selection import train_test_split
import lightning.pytorch as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

torch.manual_seed(33)

c:\Users\mdzak\anaconda3\envs\fyp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import Data

In [2]:
dna_meth = pd.read_csv('new_clean_data/merged_dna_meth.csv')
dna_meth.drop(columns='Unnamed: 0', inplace=True)
dna_meth

,CELL_LINE_NAME,1:134999,1:135191;1:135218,1:135203;1:135208,1:713376;1:713388;1:713400;1:713448;1:713450;1:713454,1:713901;1:713921;1:714178;1:714182;1:714199;1:714254;1:714258;1:714261;1:714264;1:714277;1:714278;1:714293;1:714301;1:714491;1:714511;1:714566;1:714584,1:715390;1:715392;1:715405;1:715415,1:804993;1:804999;1:805282;1:805284;1:805290;1:805327;1:805338;1:805341;1:805352;1:805445;1:805450;1:805467;1:805468,1:805474;1:805477;1:805479,1:805484;1:805486,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
0,dms53,0.7115,0.53845,0.90000,0.85480,0.0,0.95165,0.00000,0.00000,0.0000,...,-2.954484,-3.336795,4.108470,-2.624507,-3.984671,-5.033130,2.851822,0.623483,0.868145,0.428304
1,sw1116,0.5052,0.85485,0.68550,1.00000,0.0,0.00000,0.00000,0.00000,0.0000,...,-1.116696,-3.833426,2.090322,0.388698,-2.906492,4.443452,3.052729,-2.102477,0.529460,0.296894
2,umuc3,1.0000,1.00000,1.00000,0.77778,0.0,0.82652,0.02126,0.00000,0.0000,...,-4.735672,-4.698825,0.664099,0.175713,-5.240336,3.992355,2.333802,2.242455,-4.098639,-3.073182
3,hos,0.9688,1.00000,1.00000,0.78433,0.0,0.00735,0.00000,0.00000,0.0000,...,-4.630994,-5.652028,1.179937,-0.820964,-5.810339,-3.747185,1.062948,1.721562,-4.984480,-3.980797
4,rvh421,0.9367,0.91665,0.75000,0.46511,0.0,0.68032,0.00000,0.00000,0.0000,...,-4.434686,0.128226,4.261707,-0.282924,-5.240336,-0.481986,3.100707,-1.780250,-3.494500,-1.621075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,ekvx,1.0000,0.92425,0.83335,0.45908,0.0,0.40260,0.05792,0.00000,0.0000,...,-1.895873,3.783558,2.916155,-1.642778,-4.380395,1.714642,0.426465,-0.701532,-3.494500,0.375260
542,sf268,0.0645,0.50000,0.92105,0.78261,0.0,0.99325,0.00000,0.85710,0.9286,...,-3.576125,-3.336795,1.521252,0.324394,-5.938808,-4.309603,1.605002,2.190655,-4.119321,-2.587732
543,sf539,0.8846,1.00000,1.00000,0.66667,0.0,0.95085,0.00000,0.00000,0.0000,...,-4.639794,-6.386672,-0.226212,-3.370310,-5.431634,-4.142010,0.789183,-0.265838,-5.910407,-5.283113
544,snb75,0.5532,1.00000,1.00000,1.00000,0.0,0.93675,0.00000,0.11347,0.1702,...,-5.028649,-3.069203,-0.279140,-2.882799,-5.240336,-2.786002,-0.677945,1.388059,-3.494500,-1.914721


In [3]:
gene_exp = pd.read_csv('new_clean_data/merged_gene_exp.csv')
gene_exp.drop(columns='Unnamed: 0', inplace=True)
gene_exp

,CELL_LINE_NAME,ENSG00000000003.10,ENSG00000000005.5,ENSG00000000419.8,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000938.8,ENSG00000000971.11,ENSG00000001036.9,ENSG00000001084.6,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
0,22rv1,5.28,0.0,73.38,9.76,24.51,0.01,0.08,54.86,118.50,...,-4.840963,-3.112784,4.203067,-4.382138,-4.917572,-4.972312,-0.974396,1.325603,-4.384381,-3.401996
1,2313287,7.01,0.0,108.99,16.76,13.32,0.00,0.23,170.91,93.00,...,-4.584971,-3.336795,1.797167,-3.486065,-6.017003,-4.132899,3.279539,-0.040150,-4.849422,-5.303091
2,42mgba,23.09,0.0,99.28,2.73,9.27,0.02,0.45,53.29,8.36,...,-4.240733,-3.336795,-2.233603,-3.671158,-6.415517,-4.737156,-0.074308,1.621553,-4.784344,-3.927690
3,5637,57.94,0.0,98.95,3.11,31.61,0.14,1.98,71.97,9.76,...,-4.321517,-3.336795,-1.829021,-0.116397,-5.559258,-4.537337,1.818908,-0.275098,-3.494500,-5.252216
4,639v,32.02,0.0,125.99,2.69,10.45,0.00,12.89,82.50,24.13,...,-4.725271,-3.336795,1.391965,-1.822279,-4.955669,-6.639479,1.248546,1.437469,-3.986408,-3.138535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,wsudlcl2,0.74,0.0,41.71,5.60,20.21,30.57,0.46,11.78,8.32,...,-5.120747,-6.196767,-1.463565,-5.903482,-5.240336,-4.959568,-5.974346,0.641326,-7.103903,-6.196508
613,yapc,50.29,0.0,103.70,7.76,12.84,0.17,13.58,71.13,29.67,...,-1.932980,-5.076682,1.407784,1.214362,-3.897758,5.073883,1.195720,3.598733,-0.095525,2.307072
614,yh13,28.92,0.0,64.09,4.94,13.35,0.22,139.44,118.19,13.53,...,-3.595530,-3.469021,-0.787238,-2.743620,-5.957548,-3.526360,-1.963500,1.153223,-5.412392,-4.207989
615,ykg1,61.08,0.0,109.13,5.91,17.40,0.13,53.25,92.96,23.09,...,-5.262451,3.115970,0.597641,-0.763934,-5.240336,-3.225249,2.191202,0.407173,-3.494500,-2.601804


In [6]:
dna_meth_cell_list = dna_meth['CELL_LINE_NAME'].to_list()

In [9]:
gene_exp_cell_list = gene_exp['CELL_LINE_NAME'].to_list()

In [11]:
both_cell_list = list(set(dna_meth_cell_list) & set(gene_exp_cell_list))

In [14]:
consistent_dna_meth = dna_meth[dna_meth['CELL_LINE_NAME'].isin(both_cell_list)]
consistent_gene_exp = gene_exp[gene_exp['CELL_LINE_NAME'].isin(both_cell_list)]

In [19]:
consistent_dna_meth_sorted = consistent_dna_meth.sort_values('CELL_LINE_NAME')
consistent_gene_exp_sorted = consistent_gene_exp.sort_values('CELL_LINE_NAME')

In [22]:
consistent_dna_meth_sorted

,CELL_LINE_NAME,1:134999,1:135191;1:135218,1:135203;1:135208,1:713376;1:713388;1:713400;1:713448;1:713450;1:713454,1:713901;1:713921;1:714178;1:714182;1:714199;1:714254;1:714258;1:714261;1:714264;1:714277;1:714278;1:714293;1:714301;1:714491;1:714511;1:714566;1:714584,1:715390;1:715392;1:715405;1:715415,1:804993;1:804999;1:805282;1:805284;1:805290;1:805327;1:805338;1:805341;1:805352;1:805445;1:805450;1:805467;1:805468,1:805474;1:805477;1:805479,1:805484;1:805486,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
378,22rv1,0.9821,1.00000,1.00000,0.41803,0.00000,0.23610,0.00000,0.10000,0.0000,...,-4.840963,-3.112784,4.203067,-4.382138,-4.917572,-4.972312,-0.974396,1.325603,-4.384381,-3.401996
514,2313287,0.8897,0.80360,0.99105,0.65648,0.00000,0.24032,0.00000,0.00000,0.0000,...,-4.584971,-3.336795,1.797167,-3.486065,-6.017003,-4.132899,3.279539,-0.040150,-4.849422,-5.303091
352,42mgba,0.9222,0.87500,1.00000,0.49984,0.00897,0.95695,0.02262,0.12001,0.1765,...,-4.240733,-3.336795,-2.233603,-3.671158,-6.415517,-4.737156,-0.074308,1.621553,-4.784344,-3.927690
33,5637,1.0000,0.95915,1.00000,0.75370,0.00000,0.03540,0.07694,0.00000,0.0000,...,-4.321517,-3.336795,-1.829021,-0.116397,-5.559258,-4.537337,1.818908,-0.275098,-3.494500,-5.252216
527,639v,1.0000,1.00000,1.00000,0.70308,0.00000,0.16765,0.05868,0.16667,0.3000,...,-4.725271,-3.336795,1.391965,-1.822279,-4.955669,-6.639479,1.248546,1.437469,-3.986408,-3.138535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,wsudlcl2,0.7586,0.76473,1.00000,0.70492,0.00000,0.72915,0.00000,0.00000,0.0000,...,-5.120747,-6.196767,-1.463565,-5.903482,-5.240336,-4.959568,-5.974346,0.641326,-7.103903,-6.196508
236,yapc,0.9595,0.98215,1.00000,0.96413,0.00000,0.86138,0.00357,0.00000,0.0000,...,-1.932980,-5.076682,1.407784,1.214362,-3.897758,5.073883,1.195720,3.598733,-0.095525,2.307072
182,yh13,0.9211,1.00000,0.97220,0.59999,0.00000,0.94020,0.01222,0.00000,0.0000,...,-3.595530,-3.469021,-0.787238,-2.743620,-5.957548,-3.526360,-1.963500,1.153223,-5.412392,-4.207989
361,ykg1,0.8617,0.92310,0.93395,0.82007,0.00000,0.99590,0.00000,0.00000,0.0000,...,-5.262451,3.115970,0.597641,-0.763934,-5.240336,-3.225249,2.191202,0.407173,-3.494500,-2.601804


In [23]:
consistent_gene_exp_sorted

,CELL_LINE_NAME,ENSG00000000003.10,ENSG00000000005.5,ENSG00000000419.8,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000938.8,ENSG00000000971.11,ENSG00000001036.9,ENSG00000001084.6,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
0,22rv1,5.28,0.0,73.38,9.76,24.51,0.01,0.08,54.86,118.50,...,-4.840963,-3.112784,4.203067,-4.382138,-4.917572,-4.972312,-0.974396,1.325603,-4.384381,-3.401996
1,2313287,7.01,0.0,108.99,16.76,13.32,0.00,0.23,170.91,93.00,...,-4.584971,-3.336795,1.797167,-3.486065,-6.017003,-4.132899,3.279539,-0.040150,-4.849422,-5.303091
2,42mgba,23.09,0.0,99.28,2.73,9.27,0.02,0.45,53.29,8.36,...,-4.240733,-3.336795,-2.233603,-3.671158,-6.415517,-4.737156,-0.074308,1.621553,-4.784344,-3.927690
3,5637,57.94,0.0,98.95,3.11,31.61,0.14,1.98,71.97,9.76,...,-4.321517,-3.336795,-1.829021,-0.116397,-5.559258,-4.537337,1.818908,-0.275098,-3.494500,-5.252216
4,639v,32.02,0.0,125.99,2.69,10.45,0.00,12.89,82.50,24.13,...,-4.725271,-3.336795,1.391965,-1.822279,-4.955669,-6.639479,1.248546,1.437469,-3.986408,-3.138535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,wsudlcl2,0.74,0.0,41.71,5.60,20.21,30.57,0.46,11.78,8.32,...,-5.120747,-6.196767,-1.463565,-5.903482,-5.240336,-4.959568,-5.974346,0.641326,-7.103903,-6.196508
613,yapc,50.29,0.0,103.70,7.76,12.84,0.17,13.58,71.13,29.67,...,-1.932980,-5.076682,1.407784,1.214362,-3.897758,5.073883,1.195720,3.598733,-0.095525,2.307072
614,yh13,28.92,0.0,64.09,4.94,13.35,0.22,139.44,118.19,13.53,...,-3.595530,-3.469021,-0.787238,-2.743620,-5.957548,-3.526360,-1.963500,1.153223,-5.412392,-4.207989
615,ykg1,61.08,0.0,109.13,5.91,17.40,0.13,53.25,92.96,23.09,...,-5.262451,3.115970,0.597641,-0.763934,-5.240336,-3.225249,2.191202,0.407173,-3.494500,-2.601804


In [24]:
consistent_dna_meth_sorted.to_csv('new_clean_data/consistent_dna_meth.csv')
consistent_gene_exp_sorted.to_csv('new_clean_data/consistent_gene_exp.csv')